In [1]:
import json
from typing import List, Union
from pathlib import Path
import re

import pandas as pd
import requests
import yaml

In [2]:
CONFIG_FILENAME = "config_gtfs.yaml"

In [3]:
def load_csv(filepath: Union[str, Path]) -> pd.DataFrame:
    """
    Carrega o arquivo CSV e retorna um DataFrame.
    """
    return pd.read_csv(filepath)

def dump_fixtures(fixtures: List[dict], filepath: Union[str, Path]):
    """
    Salva os fixtures em um arquivo JSON.
    """
    with open(filepath, "w") as f:
        json.dump(fixtures, f, indent=4)

def resolve_e(case,record,i):
    if case == 'route_id[0:1]':
        return record['route_id'][0:1]
    if case == 'mode(1).name(4).tipo(1).versao(1)': #confirmar essa montagem de id - cruzar com o exemplo da planilha.
        return record['route_id'][0:1]+ re.sub('[^0-9]', '', record['route_short_name']).zfill(4) + record['route_id'][-1:]
    if case =='trip_id[-3:]':
        return record['trip_id'][-3:]
    if case =='i':
        return str(i)
    if case == "": #TODO: resolver o caso da via.
        return ""
    if case == "stop_id[4:5]":
        return record["stop_id"][4:5]

def gtfs_to_fixtures(dataframe, config_json, model):
    fixtures = []
    k=config_json["json"]["fields"].keys()
    e=['mode(1).name(4).tipo(1).versao(1)', 'route_id[0:1]','trip_id[-3:]', "i","","stop_id[4:5]"]
    for i, record in dataframe.iterrows():
        result = {
                "model": "pontos."+config_json["json"]["model"],
                "pk": record[config_json["json"]["pk"]] if config_json["json"]["pk"] not in e else resolve_e(config_json["json"]["pk"],record,i),
                "fields": dict.fromkeys(k,"")
            }
        for item in k:
            if config_json["json"]["fields"][item] not in e:
                result["fields"][item]=record[config_json["json"]["fields"][item]]
            else:
                result["fields"][item]=resolve_e(config_json["json"]["fields"][item],record,i)
        
            if model =="agency":
                result["pk"]=str(result["pk"])
            if model =="sequence":
                result["pk"]=int(result["pk"])
            if model =="route":
                result["fields"]['agency']=str(result["fields"]['agency'])
            if model =="trip":
                #result["fields"]['direction']=int(result["fields"]['direction'])
                if result["fields"]['headsign']!=result["fields"]['headsign']:
                    result["fields"]['headsign']=""
                
            if model =="stop":
                result["fields"]['latitude']=str(result["fields"]['latitude'])
                result["fields"]['longitude']=str(result["fields"]['longitude'])
            
                
        fixtures.append(result)
    
    dump_fixtures(fixtures, f"../fixtures/{model}.json")

In [4]:
config = yaml.load(
        open(CONFIG_FILENAME, "r"), 
        Loader=yaml.FullLoader
    )

config["models"].keys()

dict_keys(['agency', 'linha', 'route', 'trip', 'stop', 'sequence'])

In [173]:
for model in config["models"].keys():
    config_json=config["models"][model]
    dataframe = load_csv( config_json['source'] )

    gtfs_to_fixtures(dataframe, config_json, model)

In [ ]:
#TODO:
#    agency.json -- pk -- se int implica em erro -> str
#    route.json -- agency -- se int implica em erro -> str
#    trips.json -- direction -> int
#    sequence.json -- pk -- se str implica em erro -> int
#    todos  -- mode precisa ser str



In [ ]:
# generate qr-code.

In [9]:
# casos especiais:
#(caso1) id_modal>30 and id_modal<40 --> I
#(caso2) not id_modal --> J
ID_MODAL={
    'MM':'M',
    '10':'M',
    '11':'T',
    '12':'V',
    '13':'S',
    '14':'P',
    '20':'B',
    '21':'H', 
    '22':'O',
    '23':'E',
    '30':'I',
    '31 a 39':'I',
    '----':'J',
    '40':'L',
    '41':'C',
    '50':'F',
    '60':'A'
}

def translate_to_new_mode(mode):
    if mode!='MM' and int(mode)>30 and int(mode)<39:
        return "I"
    if str(mode) not in ID_MODAL:
        return "J"
    else:
        return ID_MODAL[str(mode)]
    

ID_CORREDOR={
    '0':'0',
    '1':'O',
    '2':'M',
    '3':'C',
    '4':'L',
    '5':'B',
    '6':'Z',
    '7':'Y'
}

def translate_to_new_corredor(corredor):
    return ID_CORREDOR[corredor]


TIPO_PARADA={
    '0':'0',
    '1':'C',
    '2':'A',
    '4':'P',
    '5':'E',
    '6':'S',
    '7':'U',
    '8':'X',
    '9':'T'
}
def translate_to_new_parada(parada):
    return TIPO_PARADA[parada]

print('dicionarios de para geração dos ids preparados')

def treat_results(stop,code,count,i):
    return {
            "model": "pontos.qrcode",
            "pk": count+i,
            "fields": {
                "stop": stop,
                "code": code,
            }
    }

def convert_old_to_new_stop(stop):
    return stop[0:1]+stop[3:6]+translate_to_new_mode(stop[6:8])+translate_to_new_corredor(stop[8])+stop[-6:-2]+translate_to_new_parada(stop[-2])+stop[-1:]

dicionarios de para geração dos ids preparados


In [ ]:
# Generate from api

In [175]:
import requests
last_page =True
url='https://api.mobilidade.rio/qrcode/'
fixtures=[]
i=0
while last_page:
    x = requests.get(url)
    x=json.loads(x.text)
    results=x['results']
    for item in results:
        code = item["code"]
        stop=item['stop']["id"]
        if len(stop)>12:
            stop=convert_old_to_new_stop(stop)
        fixtures.append(treat_results(stop,code,0,i))
        i+=1
    if x["next"]:
        url= x["next"]
    else:
        last_page=False

print(len(fixtures),'pontos adicionados a fixtures')

40 pontos adicionados a fixtures


In [ ]:
#add new qrcodes

In [176]:
new_code=[["1005O00198C0","1K84"],["1005O00197C0","7KKY"]]
for i in range(len(new_stops)):
        code = new_code[i][0]
        stop = new_code[i][1]
        if len(stop)>12:
            stop=convert_old_to_new_stop(stop)
        fixtures.append(treat_results(stop,code,len(fixtures),0))
print(len(fixtures),'pontos adicionados a fixtures')

42 pontos adicionados a fixtures


In [ ]:
#create json

In [177]:
dump_fixtures(fixtures, f"../fixtures/qrcode.json")

In [161]:
#generate mode.json from api

In [10]:
import requests

url='https://api.mobilidade.rio/mode/'
fixtures=[]
x = requests.get(url)
x=json.loads(x.text)
results=x['results']
for item in results:
    modal={
            "model": "pontos.mode",
            "pk": translate_to_new_mode(item["id"]),
            "fields": {
                "name": item["name"],
            }
                
    }
    fixtures.append(modal)

print(len(fixtures),'modais adicionados a fixtures')
dump_fixtures(fixtures, f"../fixtures/mode.json")

10 <class 'int'>
11 <class 'int'>
12 <class 'int'>
13 <class 'int'>
20 <class 'int'>
22 <class 'int'>
23 <class 'int'>
30 <class 'int'>
40 <class 'int'>
41 <class 'int'>
50 <class 'int'>
11 modais adicionados a fixtures
